### 유클리디안 거리공식
- 피타고라스의 정리를 기반으로 두 점사이의 거리를 계산하는 알고리즘
- 데이터 스케일에 민감하다
- 희소 데이터(Sparse)에는 잘 동작하지 않음

### 코사인 유사도
- 두 벡터의 방향이 완전히 동일하면 1, 90도의 각을 이루면 0, 180도로 반대의 방향을 가지면 -1
- 코사인 유사도는 -1 ~ 1 사이의 값을 가지며 1에 가까울수록 유사도가 높다
- 코사인 유사도는 두 벡터간의 유사도를 계산할 때 가장 널리 사용되는 방법 중 하나이다
- 코사인 유사도는 두 벡터간의 크기가 아닌 방향에 초점을 두기 때문에 벡터의 크기가 중요하지 않은 경우에 자주 사용된다
- 코사인 유사도는 두 벡터간의 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 계산한다
- 코사인 유사도는 데이터의 스케일에 영향을 받지 않는다
- 코사인 유사도는 희소 데이터에도 잘 동작한다
- 코사인 유사도는 두 벡터간의 내적을 이용하여 계산할 수 있다

In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
         return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

print(cos_sim(doc1, doc2)) # 0.6666666666666667
print(cos_sim(doc1, doc3)) # 0.6666666666666667
print(cos_sim(doc2, doc3)) # 1.0

0.6666666666666667
0.6666666666666667
1.0000000000000002


In [4]:
import pandas as pd

data = pd.read_csv('data/naver_shopping.txt', delimiter='\t', header=None)

In [6]:
data.columns = ['평점','리뷰']
data

,평점,리뷰
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,2,장마라그런가!!! 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요~


In [8]:
from eunjeon import Mecab
mecab = Mecab()

In [9]:
mecab.pos(data['리뷰'].iloc[0])

[('배공', 'NNG'), ('빠르', 'VA'), ('고', 'EC'), ('굿', 'NNG')]

In [11]:
mecab.tagset

{'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ETM': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JKB': '부사격 조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JKV': '호격 조사',
 'JX': '보조사',
 'MAG': '일반 부사',
 'MAJ': '접속 부사',
 'MM': '관형사',
 'NNB': '의존 명사',
 'NNBC': '단위를 나타내는 명사',
 'NNG': '일반 명사',
 'NNP': '고유 명사',
 'NP': '대명사',
 'NR': '수사',
 'SC': '구분자 , · / :',
 'SE': '줄임표 …',
 'SF': '마침표, 물음표, 느낌표',
 'SH': '한자',
 'SL': '외국어',
 'SN': '숫자',
 'SSC': '닫는 괄호 ), ]',
 'SSO': '여는 괄호 (, [',
 'SY': '기타 기호',
 'VA': '형용사',
 'VCN': '부정 지정사',
 'VCP': '긍정 지정사',
 'VV': '동사',
 'VX': '보조 용언',
 'XPN': '체언 접두사',
 'XR': '어근',
 'XSA': '형용사 파생 접미사',
 'XSN': '명사파생 접미사',
 'XSV': '동사 파생 접미사'}

In [13]:
# 명사와 형용사만 추출
from tqdm import tqdm
total = []
for doc in tqdm(data['리뷰']):
    rs = mecab.pos(doc)
    temp = [t[0] for t in rs if t[1] in ['NNG', 'VA']]
    total.append(' '.join(temp))

100%|██████████| 200000/200000 [00:13<00:00, 14846.53it/s]


In [15]:
# Word2Vec
from gensim.models import Word2Vec

w2v = Word2Vec(window=3,
         min_count=5,
         sg=1,
         vector_size=50,
         sentences = [doc.split(" ") for doc in total]
         )


In [16]:
w2v.wv.most_similar("배송", topn=20)

[('배소', 0.8027704358100891),
 ('도착', 0.7831131219863892),
 ('배달', 0.7441121935844421),
 ('초특급', 0.7409233450889587),
 ('추후', 0.7335488796234131),
 ('대처', 0.7327854633331299),
 ('시국', 0.731043815612793),
 ('직구', 0.7290823459625244),
 ('일주일', 0.7260314226150513),
 ('통운', 0.723820686340332),
 ('양호', 0.7230178713798523),
 ('주심', 0.7227557301521301),
 ('당시', 0.7208669185638428),
 ('제때', 0.7198733687400818),
 ('어제', 0.7180609703063965),
 ('발송', 0.7151322364807129),
 ('연휴', 0.7137156128883362),
 ('휴일', 0.7105827331542969),
 ('양품', 0.7084255814552307),
 ('재품', 0.70804363489151)]

### 긍정/부정 정답 데이터 생성

In [20]:
def labeling(row):
    if row['평점'] >= 4:
        return 1
    elif row['평점'] <= 2:
        return 0
        

In [21]:
data['label'] = data.apply(labeling, axis=1)

In [22]:
data

,평점,리뷰,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1
...,...,...,...
199995,2,장마라그런가!!! 달지않아요,0
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...,1
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요,1
199998,5,넘이쁘고 쎄련되보이네요~,1


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


In [24]:
X_train, X_test, y_train, y_test = train_test_split(total, data['label'])

In [25]:
naver_sentiment_model = Pipeline([
    ('naver_tfidf', TfidfVectorizer()),
    ('naver_lr', LogisticRegression())
])


In [26]:
naver_sentiment_model.fit(X_train, y_train)

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('naver_tfidf', TfidfVectorizer()),
                ('naver_lr', LogisticRegression())])

In [27]:
naver_sentiment_model.score(X_test, y_test)

0.7707

In [28]:
naver_sentiment_model.steps

[('naver_tfidf', TfidfVectorizer()), ('naver_lr', LogisticRegression())]

In [29]:
# 단어사전 추출
naver_vocab = naver_sentiment_model.steps[0][1].vocabulary_
# 단어 가중치 추출
naver_weight = naver_sentiment_model.steps[1][1].coef_

In [34]:
naver_df = pd.DataFrame([naver_vocab.keys(), naver_vocab.values()]).T
naver_df.sort_values(by=1, ascending=False, inplace=True)
naver_df[2] = naver_weight[0]
naver_df.columns = ['단어', '인덱스', '가중치']
naver_df

,단어,인덱스,가중치
12616,힙합,16451,0.536783
8241,힘줄,16450,-0.052675
6944,힘있,16449,0.565619
3437,힘없,16448,0.153773
10288,힘세,16447,0.486352
...,...,...,...
13651,가감,4,-0.350780
15285,가간,3,-0.594371
4307,가가,2,0.423156
15831,大王,1,-1.663468


In [35]:
naver_pos = naver_df.sort_values(by='가중치', ascending=False).head(30)

In [36]:
naver_neg = naver_df.sort_values(by='가중치', ascending=True).tail(30)

In [37]:
naver_pos

,단어,인덱스,가중치
5888,놀람,2596,5.099257
9346,내비,2383,4.962470
14406,전전날,12186,4.528673
13333,혼방,16057,4.354743
7271,연말,9809,4.198462
2615,절반,12239,3.854694
15303,생동,7208,3.829249
6258,구구,1378,3.805241
11706,화차,16145,3.695514
12782,선입견,7391,3.674003


In [38]:
naver_neg

,단어,인덱스,가중치
5619,중계,12850,2.631896
6672,미소,5085,2.650864
10593,고아,988,2.685169
9795,서하,7333,2.760710
8191,원격,10669,2.809173
5795,정말,12339,2.862476
3298,구가,1369,2.865561
9496,집시,13275,2.870828
9929,전지분유,12194,2.912320
10424,바자회,5270,2.936377


In [40]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = 'C:/Windows/Fonts/malgun.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)

In [44]:
# 필요한 라이브러리 임포트
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 설정
font_path = 'C:/Windows/Fonts/malgun.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)

# 데이터프레임 결합 및 그래프 그리기
pd.concat([naver_pos, naver_neg]).set_index('단어')['가중치'].plot(kind='bar', figsize=(20, 10))
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_13912\454252428.py", line 11, in <module>
    pd.concat([naver_pos, naver_neg]).set_index('단어')['가중치'].plot(kind='bar', figsize=(20, 10))
  File "C:\Users\USER\anaconda3\Lib\site-packages\pandas\plotting\_core.py", line 951, in __call__
    raise ValueError(
       ^^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\site-packages\pandas\plotting\_core.py", line 1947, in _get_plot_backend
  File "C:\Users\USER\anaconda3\Lib\site-packages\pandas\plotting\_core.py", line 1877, in _load_backend
    Notes
          
ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\USER\anaconda